In [ ]:
import pymssql
from getpass import getpass
from datetime import datetime
from time import sleep

# USER = input('Usuario: ')
# PASS = getpass('Senha: ')

# ORIGEM  = input('DE   (servidor.base.dbo.tabela): ')
# DESTINO = input('PARA (servidor.base.dbo.tabela): ')

# USER = input('Login: ')
# PASS = getpass('senha: ')


# CUIDADO PREENCHA CORRETAMENTE A ORIGEM DA COLETA E O DESTINO
ORIGEM = [
'SERVIDOR.BASE.SCHEMA.TABELA',
]

DESTINO = [
'SERVIDOR.BASE.SCHEMA.TABELA',
]


def insert_many(cursor, dados, tabela, colunas, step = 1000):
    sql = " insert into " + tabela + " (" + colunas + ") values "
    insert = []
    for r in range(0, len(dados), step):
        row = []
        insert = sql
        for e in dados[ r : r+step ]:
            values = []
            for n in e:
                if(type(n) == datetime):
                    values.append("'%s'" % str(n)[:23])
                else:
                    values.append("'%s'" % str(n).replace("'", "''") if str(n) != 'None' else "NULL")
            row.append("({})".format(', '.join(values)))
        insert += ', '.join(row)
        
        try:
            cursor.execute(insert)
            sleep(1)
        except Exception as e:
            ...



for ii in range(0, len(ORIGEM)):
    s_o = ORIGEM[ii].split('.')[0]
    b_o = ORIGEM[ii].split('.')[1]
    t_o = ORIGEM[ii].split('.')[3]

    s_d = DESTINO[ii].split('.')[0]
    b_d = DESTINO[ii].split('.')[1]
    t_d = DESTINO[ii].split('.')[3]

    tabelaO = '{}.dbo.{}'.format(b_o, t_o)
    tabelaD = '{}.dbo.{}'.format(b_d, t_d)

    print('''
    Atualizando Destino: {}
    A partir dos dados de Origem: {}
    '''.format(DESTINO[ii], ORIGEM[ii]))

    # conO = pymssql.connect(s_o, USER, PASS, database=b_o)
    conO = pymssql.connect(s_o, database=b_o)
    cOrigem = conO.cursor()

    # conD = pymssql.connect(s_d, USER, PASS, database=b_d)
    conD = pymssql.connect(s_d, database=b_d)
    conD.autocommit(True)
    cDestino = conD.cursor()
    

    cOrigem.execute('''select string_agg(cast(name as nvarchar(max)), ', ') from sys.columns where object_id = object_id('{}') and name not in ('rowversion', 'msrepl_synctran_ts')'''.format(t_o))
    colunas = cOrigem.fetchone()[0]
    

    cDestino.execute('''SELECT max(cast(is_identity as int)) from sys.columns where object_id = object_id('{}')'''.format(t_d))
    identity = cDestino.fetchone()[0]

    
    cOrigem.execute('''
    SELECT STRING_AGG(COLUMN_NAME, ',') WITHIN GROUP (ORDER BY ORDINAL_POSITION) AS pks
    FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
    WHERE OBJECTPROPERTY(OBJECT_ID(CONSTRAINT_SCHEMA + '.' + QUOTENAME(CONSTRAINT_NAME)), 'IsPrimaryKey') = 1
    AND TABLE_NAME = '{}';'''.format(t_o))
    pks = cOrigem.fetchone()[0]

    if not pks:
        pks = colunas


    print('Ajustando FKS...')
    cDestino.execute('''
    SELECT coalesce(object_name(fkc.parent_object_id), object_name(obj.parent_object_id)) tabela, obj.name, obj.type
    FROM sys.foreign_key_columns fkc
    right join sys.objects obj on fkc.constraint_object_id = obj.object_id
    where  '{}' in ( object_name(fkc.parent_object_id), object_name(fkc.referenced_object_id),
    object_name(obj.parent_object_id) )
    AND obj.type in ('F', 'C', 'TR')
    '''.format(t_d))
    fks = cDestino.fetchall()


    for fk in fks:
        if(fk[2] == 'TR'):
            cDestino.execute('''DISABLE TRIGGER {} ON {};'''.format(fk[1], fk[0]))
        else:
            cDestino.execute('''ALTER TABLE {} NOCHECK CONSTRAINT {}; '''.format(fk[0], fk[1]))

    cDestino.execute('''EXEC sp_configure 'max text repl size (B)', -1''')
    cDestino.execute('RECONFIGURE WITH OVERRIDE')

    if identity:
        if(identity > 0):
            cDestino.execute('SET IDENTITY_INSERT {} ON'.format(t_d))

    cOrigem.execute('SELECT COUNT(*) FROM ' + tabelaO)
    total_rows = cOrigem.fetchone()[0]
    batch_size = 100000
    iniciar_em = 0

    if iniciar_em == 0:
        print('Deletando Dados no Destino..')
        cDestino.execute('DELETE FROM '+tabelaD)

    for offset in range(iniciar_em, total_rows, batch_size):
        print('Selecionando Dados..')
        cOrigem.execute('''SELECT {} FROM {} ORDER BY {} 
                        OFFSET {} ROWS FETCH NEXT {} ROWS ONLY;'''.format(colunas, tabelaO, pks, offset, batch_size))
        dados = cOrigem.fetchall()

        print('Inserindo dados...')
        insert_many(cDestino, dados, tabelaD, colunas)

        current_records = min(offset + batch_size, total_rows)
        print(f"Registros inseridos: {current_records} / {total_rows}")
        del dados


    if(identity > 0):
        cDestino.execute('SET IDENTITY_INSERT {} OFF'.format(t_d))

    cDestino.execute('''EXEC sp_configure 'max text repl size (B)', 65536''')
    cDestino.execute('RECONFIGURE WITH OVERRIDE')

    print('Reajustando FKS...')
    for fk in fks:
        if(fk[2] == 'TR'):
            cDestino.execute('''ENABLE TRIGGER {} ON {};'''.format(fk[1], fk[0]))
        else:
            cDestino.execute('''ALTER TABLE {} CHECK CONSTRAINT {}; '''.format(fk[0], fk[1]))

    print('Tabela Atualizada')